In [30]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 
%matplotlib inline

import examples_lib

In [48]:
a, b, x = examples_lib.get_quadratic_params(3)
examples_lib.get_quadratic_obj(x, a, b)

<tf.Tensor: shape=(), dtype=float64, numpy=1.6189196288206213>

In [3]:
mu, sigma, x, w = examples_lib.get_normal_params(10)
examples_lib.get_norm_ll_infl(x, mu, sigma, w)

<tf.Tensor: shape=(), dtype=float32, numpy=-10.754641>

In [4]:
x, mu_vec, sigma_vec, pi_vec, z_true = examples_lib.get_norm_cluster_params(3, 10)
e_z = z_true * 0.8 + 0.1
examples_lib.get_norm_clustering_mstep(x, mu_vec, sigma_vec, pi_vec, e_z)

<tf.Tensor: shape=(), dtype=float64, numpy=-1347.8231145018508>

In [5]:
theta, x_mat, y = examples_lib.get_logistic_params(10, 3)
examples_lib.get_logistic_ll(y, x_mat, theta)

<tf.Tensor: shape=(), dtype=float64, numpy=0.551107393829309>

## Forward mode example

In [52]:
dim = 3
a, b, x = examples_lib.get_quadratic_params(dim)
def obj(x):
    return examples_lib.get_quadratic_obj(x, a, b)
xopt = tf.squeeze(-tf.linalg.solve(a, tf.expand_dims(b, 1)))
xopt

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([-1.12277677, -6.81905134,  2.57167325])>

In [114]:
def get_tf_args(*argv):
    tf_argv = []
    for arg in argv:
        tf_argv.append(tf.Variable(arg))
    return tf_argv

def get_tf_fun(fun):
    def tf_fun(*argv):
        tf_argv = get_tf_args(*argv)
        return fun(*tf_argv)
    return tf_fun

def get_gradient_fun(fun):
    def fun_grad(*argv):
        tf_argv = get_tf_args(*argv)
        with tf.GradientTape() as tape:
            result = fun(*tf_argv)
        return tape.gradient(result, tf_argv)
    return fun_grad
        
x = np.random.random(dim)
print(get_tf_args(x))
print(get_tf_fun(obj)(x))
get_obj_grad = get_gradient_fun(obj)
print(get_obj_grad(x))
print(get_obj_grad(xopt))

[<tf.Variable 'Variable:0' shape=(3,) dtype=float64, numpy=array([0.35674624, 0.64522893, 0.48022861])>]
tf.Tensor(0.9982161360979686, shape=(), dtype=float64)
[<tf.Tensor: shape=(3,), dtype=float64, numpy=array([1.30662422, 0.61172545, 1.6995016 ])>]
[<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2.22044605e-16, 2.22044605e-16, 3.33066907e-16])>]


In [116]:
# Note that this doesn't work.
x_tf = get_tf_args(x)[0]
with tf.GradientTape() as tape:
    grad = get_obj_grad(x_tf)[0]
hess = tape.jacobian(grad, x_tf)
print(grad)
print(hess)

tf.Tensor([1.30662422 0.61172545 1.6995016 ], shape=(3,), dtype=float64)
None


In [123]:
# Hessian that works.
x_tf = get_tf_args(x)[0]
with tf.GradientTape() as tape1:
    with tf.GradientTape() as tape2:
        obj_val = obj(x_tf)
    grad = tape2.gradient(obj_val, x_tf)
hess = tape1.jacobian(grad, x_tf)
print(hess, hess - a)

tf.Tensor(
[[1.18685237 0.26360026 1.15563059]
 [0.26360026 0.15110692 0.43328023]
 [1.15563059 0.43328023 1.55127487]], shape=(3, 3), dtype=float64) tf.Tensor(
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]], shape=(3, 3), dtype=float64)


In [118]:
# This works
x_tf = tf.constant(x) # Can also be tf.Variable
with tf.autodiff.ForwardAccumulator(x_tf, grad) as accum:
    obj_val = obj(x_tf)

forward_grad_grad = accum.jvp(obj_val)    
print(forward_grad_grad, forward_grad_grad - tf.tensordot(grad, grad, 1))    

tf.Tensor(4.969780573018578, shape=(), dtype=float64) tf.Tensor(8.881784197001252e-16, shape=(), dtype=float64)


In [131]:
# This works
obj_grad = get_obj_grad(x_tf)[0]
v = tf.Variable(obj_grad)
x_tf = tf.Variable(x)
with tf.autodiff.ForwardAccumulator(x_tf, v) as accum:
    with tf.GradientTape() as tape:
        obj_val = obj(x_tf)
    grad = tape.gradient(obj_val, x_tf)

forward_hvp = accum.jvp(grad)

print(forward_hvp)
print(tf.tensordot(hess, grad, axes=1))

tf.Tensor([3.67601708 1.17322288 4.41141758], shape=(3,), dtype=float64)
tf.Tensor([3.67601708 1.17322288 4.41141758], shape=(3,), dtype=float64)


In [132]:
# This works, though it is not in the right order.
obj_grad = get_obj_grad(x_tf)[0]
v = tf.Variable(obj_grad)
x_tf = tf.Variable(x)
with tf.GradientTape() as tape:
    with tf.autodiff.ForwardAccumulator(x_tf, v) as accum:
        obj_val = obj(x_tf)
    forward_gvp = accum.jvp(obj_val)
forward_hvp = tape.gradient(forward_gvp, x_tf)

print(forward_hvp)
print(tf.tensordot(hess, grad, axes=1))

tf.Tensor([3.67601708 1.17322288 4.41141758], shape=(3,), dtype=float64)
tf.Tensor([3.67601708 1.17322288 4.41141758], shape=(3,), dtype=float64)
